## Checkpoint 3 - PLN

Fabio Rocha de Oliveira Lanfranchi - 87226

### Problemática

Nosso objetivo neste notebook é desenvolver um sistema que seja capaz de fazer recomendações para usuários da plataforma Crunchyroll. Essa recomendação será feita utilizando como base as informações de notas que os usuários dão para os animes assistidos.

Pensando neste problema, desenvolveremos um sistema de recomendação que utilizará o algoritmo K-NearestNeighbors, que, com referencia da menor distancia euclidiana entre os pontos, retornará para o usuário sugestões de séries. 

### Importação dos dados e análise exploratória

In [1]:
import pandas as pd

Serão utilizadas duas bases de dados, uma contendo os títulos e outra contendo as notas que os usuários forneceram para esses títulos.

In [2]:
# Alocando dfs
df_title = pd.read_csv('C:\\Users\\f_lan\\Downloads\\anime_id_titles.csv.gz', compression='gzip')
df_rating = pd.read_csv('C:\\Users\\f_lan\\Downloads\\anime_data.csv.gz', compression='gzip')

In [3]:
# Printando primeiras linhas do df de titulos
df_title.head()

,item_id,title
0,10,The Prince of Tennis
1,100,Neon Genesis Evangelion
2,1000,Full Metal Panic! The Second Raid
3,10000,22/7: Shampoo no Nioi ga Shita
4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance


In [4]:
# Printando primeiras linhas do df com as notas dos usuários
df_rating.head()

,user_id,item_id,rating
0,0,147,5.0
1,1,1512,4.5
2,1,599,4.0
3,1,2292,4.5
4,1,1078,3.0


In [5]:
# Verificando shape dos dfs
print (df_title.shape)
print (df_rating.shape)

(16621, 2)
(8765945, 3)


### Preparação dos dados

Vamos concatenar as bases de dados, fazendo um inner join na coluna item_id

In [6]:
# Concatenando os dfs
df = pd.merge(df_title, df_rating, on= 'item_id')
df.head()

,item_id,title,user_id,rating
0,10,The Prince of Tennis,34,4.0
1,10,The Prince of Tennis,106,5.0
2,10,The Prince of Tennis,132,4.5
3,10,The Prince of Tennis,212,4.5
4,10,The Prince of Tennis,247,4.5


A seguir, pivotaremos o dataset para conseguir uma visão das notas individuais de cada usuário para cada série de anime. Para as séries que não tiverem votos por determinado usuário assumiremos o valor 0.

In [7]:
# Pivotando o df final
df_pivot = df.pivot_table(columns='title', index='user_id', values='rating').fillna(0)
df_pivot.head(10)

title,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiji""","""Eikou Naki Tensai-tachi"" Kara no Monogatari","""Eiyuu"" Kaitai","""Kiss Dekiru Gyoza"" x Mameshiba Movie","""Parade"" de Satie","""hitman"" bang: I=Fantasy",(K)NoW_NAME: Harvest,...,xxxHOLiC Shunmuki,y0c1e: To Dimension,yama: Arui wa Eiga no You na,yama: Masshiro,your name.,yourness: Kago no Naka ni Tori,zap dog,Üks Uks,ēlDLIVE,∞
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


### Escolha do algoritmo e treinamento dos dados

Como temos muitos dados, vamos criar uma matriz esparsa que facilitará o treinamento dos dados

In [8]:
from scipy.sparse import csr_matrix
df_sparse = csr_matrix(df_pivot)

Para o nosso estudo, utilizaremos o algoritmo NearestNeighbors

In [9]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors()
knn.fit(df_sparse)

NearestNeighbors()

### Verificando resultados

A seguir iremos testar a máquina preditiva. Para isso, pegamos um usuário ao acaso, neste exemplo, o usuário 10

In [10]:
# Testando

index_user = 10
dist, index_ngbr = knn.kneighbors(df_pivot.iloc[index_user].values.reshape(1,-1), n_neighbors=3)

In [11]:
# Verificando a distancia entre os vizinhos
dist

array([[ 0.        , 64.75916923, 64.86717197]])

In [12]:
# Verificando quais são os indices dos usuários vizinhos
index_ngbr

array([[   10, 50974, 11415]], dtype=int64)

In [13]:
# Comparando resultados
user = df_pivot.index[index_user]

for i in range(0, len(dist.flatten())):
    if i == 0:
        print (f'Usuário: {user}')
    else:
        print (f'Vizinho {df_pivot.index[index_ngbr.flatten()[i]]} com distancia de {dist.flatten()[i]}')

Usuário: 10
Vizinho 59265 com distancia de 64.75916923494309
Vizinho 13213 com distancia de 64.8671719747362


Ou seja, vamos pegar o usuário mais proximo (nesse caso, o 59265), e utilizaremos as suas preferencias para fazer sugestões para o usuário 10

### Comparação de usuário target com K vizinho mais próximo

In [14]:
# Criando df para o usuário 10
df_compara = df_pivot.loc[user].to_frame()

# Criando df para o usuário 59265
viz_prox = df_pivot.index[index_ngbr.flatten()[1]]
ds_vizinho = df_pivot.loc[viz_prox].to_frame()

Dataset contendo as notas postadas pelo usuário 10

In [15]:
df_compara

,10
title,
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0
"""Bungaku Shoujo"" Memoire",0.0
"""Bungaku Shoujo"" Movie",0.0
"""Eiji""",0.0
"""Eikou Naki Tensai-tachi"" Kara no Monogatari",0.0
...,...
yourness: Kago no Naka ni Tori,0.0
zap dog,0.0
Üks Uks,0.0


Dataset contendo as notas postadas pelo usuário 59265

In [16]:
ds_vizinho

,59265
title,
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0
"""Bungaku Shoujo"" Memoire",0.0
"""Bungaku Shoujo"" Movie",0.0
"""Eiji""",0.0
"""Eikou Naki Tensai-tachi"" Kara no Monogatari",0.0
...,...
yourness: Kago no Naka ni Tori,0.0
zap dog,0.0
Üks Uks,0.0


Abaixo, visualizaremos uma comparação das notas entre os usuários 10 e 59265

In [17]:
ds_titulos = pd.merge(df_compara, ds_vizinho, on='title').sort_values(by=viz_prox, ascending=False)
ds_titulos

,10,59265
title,,
Kill Bill Chapter 3: The Origin of O-Ren,0.0,5.0
The Girl Who Leapt Through Time,3.5,5.0
Fullmetal Alchemist The Movie: Conqueror of Shamballa,3.5,5.0
The Disappearance of Haruhi Suzumiya,4.0,5.0
Cowboy Bebop,4.5,5.0
...,...,...
Hi-sCool! Seha Girls Special,0.0,0.0
HiGH&LOW g-sword,0.0,0.0
HiGH&LOW g-sword OVA,0.0,0.0


Removendo as séries já assistidas pelo usuário 10, ou seja as que possuem nota, e deixando apenas as que têm postagem do usuário 59265 

In [18]:
ds_titulos = ds_titulos[(ds_titulos[viz_prox] > 0) & (ds_titulos[user]==0)].reset_index()
ds_titulos

,title,10,59265
0,Kill Bill Chapter 3: The Origin of O-Ren,0.0,5.0
1,Gargantia on the Verdurous Planet,0.0,4.5
2,Neon Genesis Evangelion: The End of Evangelion,0.0,4.0
3,Silver Spoon,0.0,4.0
4,Anohana: The Flower We Saw That Day,0.0,4.0
5,Level E,0.0,3.5
6,The Eccentric Family,0.0,3.0


Retornando o top 5 séries com nota alta do vizinho mais próximo

In [19]:
# Recomendando os 5 animes com as maiores notas baseadas no vizinho
ds_titulos.title[:5]

0          Kill Bill Chapter 3: The Origin of O-Ren
1                 Gargantia on the Verdurous Planet
2    Neon Genesis Evangelion: The End of Evangelion
3                                      Silver Spoon
4               Anohana: The Flower We Saw That Day
Name: title, dtype: object

### Conclusão

Nosso objetivo foi apresentar um algoritmo de fácil implementação e que fosse efetivo. Ainda que pudessemos efetuar muitas melhorias em nosso algoritmo, efetuando um melhor ajuste nos hiperparametros atraves de um fine-tunning, ainda assim o algoritmo KNN foi capaz de retornar títulos coerentes com os assistidos pelo nosso usuário alvo. Ao efetuar uma busca pelos titulos percebemos que o usuário 10 deu notas para séries de gêneros váriados, como Ação, Ficção científica e Romance, e seriados com gêneros semelhantes foram ofertados como sugestão.

### Referencias

COMO CRIAR SISTEMAS DE RECOMENDAÇÃO: 
https://www.youtube.com/watch?v=9tAU8fRgq2g&ab_channel=Ci%C3%AAnciadosDados

Introdução a Sistemas de Recomendação - KNN: 
https://www.youtube.com/watch?v=Vkh2ay8hbbo&ab_channel=labbme_in
